# Volatility Prediction Model!

In [ ]:
pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!mv /path/to/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat '/path/to/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sachinshakya05","key":"8194891084ca0cefcbfb2f649d641a17"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c nk-iv-prediction
!unzip nk-iv-prediction.zip

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 4, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 6, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 434, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open nk-iv-prediction.zip, nk-iv-prediction.zip.zip or nk-iv-prediction.zip.ZIP.


In [ ]:
import pandas as pd

train = pd.read_parquet("train_data.parquet")
test = pd.read_parquet("test_data.parquet")

train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train_data.parquet'

In [ ]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(train.shape)
print(train.columns[:10])
print(train.isna().sum().sort_values(ascending=False).head(10))

(178340, 97)
Index(['timestamp', 'underlying', 'expiry', 'call_iv_23500', 'call_iv_23600',
       'call_iv_23700', 'call_iv_23800', 'call_iv_23900', 'call_iv_24000',
       'call_iv_24100'],
      dtype='object')
timestamp        0
underlying       0
expiry           0
call_iv_23500    0
call_iv_23600    0
call_iv_23700    0
call_iv_23800    0
call_iv_23900    0
call_iv_24000    0
call_iv_24100    0
dtype: int64


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

In [ ]:
# 1. Load the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (178340, 97)
Test shape: (12065, 96)


In [ ]:
# 2. Identify IV columns and feature columns
iv_cols = [col for col in train.columns if "iv" in col]
feature_cols = [col for col in train.columns if col.startswith("X")]

X_train = train[feature_cols]
y_train = train[iv_cols]
X_test = test[feature_cols]

In [ ]:

# 3. Handle missing values (fill with median or 0)
imputer = SimpleImputer(strategy="median")
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=feature_cols)
X_test = pd.DataFrame(imputer.transform(X_test), columns=feature_cols)

In [ ]:
# 4. Train Random Forest models per IV column
models = {}
preds = pd.DataFrame(index=test.index)

for col in iv_cols:
    print(f"Training model for: {col}")
    notna_mask = ~y_train[col].isna()

    # Skip columns with too few valid values
    if notna_mask.sum() < 10:
        print(f"Skipping {col} due to insufficient non-null values.")
        continue

    y = y_train.loc[notna_mask, col]
    X = X_train.loc[notna_mask]

    model = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=42)
    model.fit(X, y)
    models[col] = model

    preds[col] = model.predict(X_test)

Training model for: call_iv_23500
Training model for: call_iv_23600
Training model for: call_iv_23700
Training model for: call_iv_23800
Training model for: call_iv_23900
Training model for: call_iv_24000
Training model for: call_iv_24100
Training model for: call_iv_24200
Training model for: call_iv_24300
Training model for: call_iv_24400
Training model for: call_iv_24500
Training model for: call_iv_24600
Training model for: call_iv_24700
Training model for: call_iv_24800
Training model for: call_iv_24900
Training model for: call_iv_25000
Training model for: call_iv_25100
Training model for: call_iv_25200
Training model for: call_iv_25300
Training model for: call_iv_25400
Training model for: call_iv_25500
Training model for: call_iv_25600
Training model for: call_iv_25700
Training model for: call_iv_25800
Training model for: call_iv_25900
Training model for: call_iv_26000
Training model for: put_iv_22500
Training model for: put_iv_22600
Training model for: put_iv_22700
Training model fo

In [ ]:
# 5. Create submission file
submission = sample.copy()
for col in iv_cols:
    if col in submission.columns and col in preds.columns:
        submission[col] = preds[col]

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv created.")

✅ submission.csv created.
